In [5]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [6]:
directory = '/projects/ps-yeolab/mstp_bootcamp_2016/'
counts = pd.read_csv(directory+"featureCounts.txt",sep="\t",index_col=0,
                    comment="#")
print counts.shape
counts.head()

(57820, 10)


,Chr,Start,End,Strand,Length,/home/ucsd-train01/scratch/annotations/allchrom.fa,lin28b_ctrl_rep1.bam,lin28b_ctrl_rep2.bam,lin28b_kd_rep1.bam,lin28b_kd_rep2.bam
Geneid,,,,,,,,,,
ENSG00000223972.4,chr1;chr1;chr1;chr1,11869;12595;12975;13221,12227;12721;13052;14412,+;+;+;+,1756,0,0,2,0,0
ENSG00000227232.4,chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;c...,14363;14970;15796;16607;16854;17233;17498;1760...,14829;15038;15947;16765;17055;17368;17504;1774...,-;-;-;-;-;-;-;-;-;-;-;-;-,2073,0,294,294,334,410
ENSG00000243485.2,chr1;chr1;chr1,29554;30267;30976,30039;30667;31109,+;+;+,1021,0,0,0,0,0
ENSG00000237613.2,chr1;chr1;chr1,34554;35245;35721,35174;35481;36081,-;-;-,1219,0,0,2,4,7
ENSG00000268020.2,chr1;chr1,52473;54830,53312;54936,+;+,947,0,0,0,0,0


In [7]:
counts.drop(['Chr','Start','End','Strand',
              '/home/ucsd-train01/scratch/annotations/allchrom.fa'], 
           axis=1, inplace=True)
counts.head()

,Length,lin28b_ctrl_rep1.bam,lin28b_ctrl_rep2.bam,lin28b_kd_rep1.bam,lin28b_kd_rep2.bam
Geneid,,,,,
ENSG00000223972.4,1756,0,2,0,0
ENSG00000227232.4,2073,294,294,334,410
ENSG00000243485.2,1021,0,0,0,0
ENSG00000237613.2,1219,0,2,4,7
ENSG00000268020.2,947,0,0,0,0


In [9]:
# save_directory = '/home/ucsd-train01/processed_data/'
# counts.to_csv(save_directory+'counts_for_deseq2.csv')

In [10]:
conditions = pd.DataFrame(index = counts.columns[1:])
conditions.head()

""
lin28b_ctrl_rep1.bam
lin28b_ctrl_rep2.bam
lin28b_kd_rep1.bam
lin28b_kd_rep2.bam


In [11]:
features= ['control','control','knockdown','knockdown']
conditions['condition'] = features
conditions.head()

,condition
lin28b_ctrl_rep1.bam,control
lin28b_ctrl_rep2.bam,control
lin28b_kd_rep1.bam,knockdown
lin28b_kd_rep2.bam,knockdown


In [ ]:
conditions.to_csv(save_directory+"conditions_matrix_deseq2.csv")

In [12]:
def counts_to_rpkm(featureCountsTable):
    counts = featureCountsTable.ix[:,1:]
    lengths = featureCountsTable['Length']
    mapped_reads = counts.sum()
    return (counts * pow(10,9)).div(mapped_reads, axis=1).div(lengths, axis=0)

In [15]:
rpkm = counts_to_rpkm(counts)
print rpkm.shape
rpkm.head()

(57820, 4)


,lin28b_ctrl_rep1.bam,lin28b_ctrl_rep2.bam,lin28b_kd_rep1.bam,lin28b_kd_rep2.bam
Geneid,,,,
ENSG00000223972.4,0.000000,0.062309,0.000000,0.000000
ENSG00000227232.4,9.087219,7.758734,10.038027,8.460869
ENSG00000243485.2,0.000000,0.000000,0.000000,0.000000
ENSG00000237613.2,0.000000,0.089757,0.204436,0.245655
ENSG00000268020.2,0.000000,0.000000,0.000000,0.000000


In [16]:
rpkm_filtered = rpkm.loc[rpkm.mean(axis=1) > 1]
print rpkm_filtered.shape
rpkm_filtered.head()

(11973, 4)


,lin28b_ctrl_rep1.bam,lin28b_ctrl_rep2.bam,lin28b_kd_rep1.bam,lin28b_kd_rep2.bam
Geneid,,,,
ENSG00000227232.4,9.087219,7.758734,10.038027,8.460869
ENSG00000238009.2,1.543956,1.992690,2.391527,1.953761
ENSG00000237683.5,18.444499,24.506101,35.775001,33.052831
ENSG00000239906.1,2.182092,1.863086,4.822127,5.959920
ENSG00000241860.2,1.996177,0.953730,1.267157,0.987473
